In [72]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans

In [61]:
df = pd.read_csv("../raw_data/preprocessed_data.csv", index_col=0)

df.head()

,description,points,price,province,title,winery,variety_adj,type,region
country,,,,,,,,,
Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Quinta dos Avidagos,portuguese red,red,Douro
US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Oregon,Rainstorm 2013 Pinot Gris (Willamette Valley),Rainstorm,pinot gris,white,Willamette Valley
US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Michigan,St. Julian 2013 Reserve Late Harvest Riesling ...,St. Julian,riesling,white,Lake Michigan Shore
US,"Much like the regular bottling from 2012, this...",87,65.0,Oregon,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Sweet Cheeks,pinot noir,red,Willamette Valley
Spain,Blackberry and raspberry aromas show a typical...,87,15.0,Northern Spain,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tandem,tempranillo-merlot,red,Navarra


In [94]:
X=df.drop(columns=['description','title'])
X.reset_index(inplace=True)

In [70]:
X

,country,points,price,province,winery,variety_adj,type,region
0,Portugal,87,15.0,Douro,Quinta dos Avidagos,portuguese red,red,Douro
1,US,87,14.0,Oregon,Rainstorm,pinot gris,white,Willamette Valley
2,US,87,13.0,Michigan,St. Julian,riesling,white,Lake Michigan Shore
3,US,87,65.0,Oregon,Sweet Cheeks,pinot noir,red,Willamette Valley
4,Spain,87,15.0,Northern Spain,Tandem,tempranillo-merlot,red,Navarra
...,...,...,...,...,...,...,...,...
111529,Germany,90,28.0,Mosel,Dr. H. Thanisch (Erben MÃ¼ller-Burggraef),riesling,white,Mosel
111530,US,90,75.0,Oregon,Citation,pinot noir,red,Oregon
111531,France,90,30.0,Alsace,Domaine Gresser,gewurztraminer,white,Alsace
111532,France,90,32.0,Alsace,Domaine Marcel Deiss,pinot gris,white,Alsace


In [95]:
label_encoder_country = LabelEncoder()
label_encoder_province = LabelEncoder()
label_encoder_winery = LabelEncoder()
label_encoder_variety = LabelEncoder()
label_encoder_region = LabelEncoder()
one_hot_encoder_type = OneHotEncoder(sparse=False)

X['country'] = label_encoder_country.fit_transform(X['country'])
X['province'] = label_encoder_province.fit_transform(X['province'])
X['winery'] = label_encoder_winery.fit_transform(X['winery'])
X['variety_adj'] = label_encoder_variety.fit_transform(X['variety_adj'])
X['region'] = label_encoder_region.fit_transform(X['region'])

type_encoded = one_hot_encoder_type.fit_transform(X[['type']])
type_categories = one_hot_encoder_type.get_feature_names_out(['type'])
type_encoded=pd.DataFrame(type_encoded)
X[type_categories] = type_encoded
X.drop(columns=['type'], inplace=True)
X

/Users/felipedalcin/.pyenv/versions/3.10.6/envs/WineGen/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,country,points,price,province,winery,variety_adj,region,type_red,type_rosé,type_sparkling,type_white
0,30,87,15.0,106,12188,441,486,1.0,0.0,0.0,0.0
1,39,87,14.0,265,12248,427,1554,0.0,0.0,0.0,1.0
2,39,87,13.0,215,13564,470,696,0.0,0.0,0.0,1.0
3,39,87,65.0,265,13788,431,1554,1.0,0.0,0.0,0.0
4,36,87,15.0,259,13868,578,955,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
111529,16,90,28.0,229,6125,470,882,0.0,0.0,0.0,1.0
111530,39,90,75.0,265,4411,431,999,1.0,0.0,0.0,0.0
111531,14,90,30.0,11,5493,205,32,0.0,0.0,0.0,1.0
111532,14,90,32.0,11,5579,427,32,0.0,0.0,0.0,1.0


In [133]:
kmeans = KMeans(n_clusters=5600, random_state=0, n_init="auto").fit(X)

In [134]:
# new_X = X.sample(5)


pred = kmeans.predict(X)

In [135]:
pred=pd.DataFrame(pred)
pred.value_counts()

1821    237
1957    223
4195    175
96      156
71      135
       ... 
1294      1
311       1
5444      1
842       1
1420      1
Name: count, Length: 5600, dtype: int64

In [137]:
df.iloc[pred[pred[0]==20].index]

,description,points,price,province,title,winery,variety_adj,type,region
country,,,,,,,,,
Germany,"Crisp, pristine peach and apricot notes resona...",91,16.0,Rheinhessen,Weingut Liebfrauenstift 2014 Dry Riesling (Rhe...,Weingut Liebfrauenstift,riesling,white,Rheinhessen
Germany,"Intensely fragrant, this honey-sweet auslese w...",95,66.0,Rheingau,Weingut Hans Bausch 2011 Hattenheimer Hassel A...,Weingut Hans Bausch,riesling,white,Rheingau
US,"The wine was aged in 30% new American oak, and...",85,32.0,Oregon,Weisinger 2013 Estate Tempranillo (Rogue Valley),Weisinger,tempranillo,red,Rogue Valley
US,Like its companion Syrah from Cedar Ridge Vine...,85,26.0,Oregon,Weisinger 2012 Meadowlark Vineyard Syrah (Rogu...,Weisinger,syrah,red,Rogue Valley
Germany,"While aromatically subdued, this semidry, late...",90,18.0,Rheinhessen,Weingut Georg Albrecht Schneider 2013 Nierstei...,Weingut Georg Albrecht Schneider,riesling,white,Rheinhessen
Portugal,Toffee and vanilla aromas give an intensely sw...,91,125.0,Port,Wiese & Krohn 1964 Branco Colheita (Port),Wiese & Krohn,port,red,Port
Portugal,"Mature, definitely dry in character, with attr...",91,16.0,Port,Wiese & Krohn NV Dry White (Port),Wiese & Krohn,port,red,Port
US,This blend of Cinsault (75%) and MourvÃ¨dre al...,83,23.0,Washington,Vortex 2010 CM2 Red (Rattlesnake Hills),Vortex,red blend,red,Rattlesnake Hills
Germany,"Intensely aromatic, intensely fruity, this Rie...",88,18.0,Rheinhessen,Weingut Georg Albrecht Schneider 2011 Nierstei...,Weingut Georg Albrecht Schneider,riesling,white,Rheinhessen
